In [11]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [1]:
#smile file generation for actives
def activeSmiles(fname):
    actives = open('/Users/dshrestha/Desktop/Pharos/Actives/'+fname+'.txt','r').readlines()
    active_smiles = open('/Users/dshrestha/Desktop/Pharos/smiles/active_'+fname+'_smiles.smi','w')
    
    for lines in actives:
        line = lines.split('\t')[0]
        active_smiles.write(line+'\n')

    active_smiles.close()
#activeSmiles('Acetyl_coenzyme_A_transporter_1')

In [2]:
#smile file generation for decoys
def decoySmiles(fname):
    decoy_path ='/Users/dshrestha/Desktop/Pharos/Decoys_set_2/'+fname+'/decoys/'
    files = os.listdir(decoy_path)
    decoy_smiles = open('/Users/dshrestha/Desktop/Pharos/smiles/decoy_'+fname+'_smiles.smi','w')
    for file in files:
        if file != '.DS_Store':
            decoys = open(decoy_path+file,'r').readlines() 
            for lines in decoys:
                if not lines.startswith('ligand'):
                    line = lines.split('\t')[0]
                    decoy_smiles.write(line+'\n')
                else:
                    decoy_smiles.write(lines.split('\t')[1]+'\n')
    decoy_smiles.close()
#decoySmiles('Acetyl_coenzyme_A_transporter_1')

In [14]:
#converting smiles to sdf
def smiles2sdf(fname):
    os.system('python /Users/dshrestha/Downloads/mayachemtools/bin/RDKitConvertFileFormat.py -i /Users/dshrestha/Desktop/Pharos/smiles/active_'+fname+'_smiles.smi'+' --ov -o /Users/dshrestha/Desktop/Pharos/sdf/active_'+fname+'_sdf.sdf')
    print('generated sdf for actives')
    os.system('python /Users/dshrestha/Downloads/mayachemtools/bin/RDKitConvertFileFormat.py -i /Users/dshrestha/Desktop/Pharos/smiles/decoy_'+fname+'_smiles.smi'+' --ov -o /Users/dshrestha/Desktop/Pharos/sdf/decoy_'+fname+'_sdf.sdf')
    print('generated sdf for decoys')
#smiles2sdf('Acetyl_coenzyme_A_transporter_1')



In [5]:
#generating features for actives
def TPATF(fname):
    command='perl  /Users/dshrestha/Downloads/mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl -r /Users/dshrestha/Desktop/Pharos/fingerprints/actives_'+fname+' --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o /Users/dshrestha/Desktop/Pharos/sdf/active_'+fname+'_sdf.sdf'
    os.system(command)
    print('generated fingerprint for actives')
    #fingerprint generation for decoys
    command2='perl  /Users/dshrestha/Downloads/mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl -r /Users/dshrestha/Desktop/Pharos/fingerprints/decoys_'+fname+' --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o /Users/dshrestha/Desktop/Pharos/sdf/decoy_'+fname+'_sdf.sdf'
    os.system(command2)
    print('generated fingerprint for decoys')
    
    

In [7]:
#creating dataframe of features and labels for actives and decoys
def getTPATF(fname):
    print('processing actives features')
    active = open('/Users/dshrestha/Desktop/Pharos/fingerprints/actives_'+fname+'.csv','r').readlines()
    frame1=[]
    for lines in active:
        #print(lines)
        if 'Cmpd' in lines:
            line = lines.split(';')[5].rstrip('"\n').split(' ')
            #print(len(line))
            df = pd.DataFrame(np.array(line).reshape(1,len(line)))
            df.astype(int)
            frame1.append(df)
    active_val = [1]*len(frame1)
    
    print('processing decoys features')
    decoy = open('/Users/dshrestha/Desktop/Pharos/fingerprints/decoys_'+fname+'.csv','r').readlines()
    frame2 =[]
    for lines in decoy:
        #print(lines)
        if 'Cmpd' in lines:
            line = lines.split(';')[5].rstrip('"\n').split(' ')
            #print(len(line))
            df = pd.DataFrame(np.array(line).reshape(1,len(line)))
            df.astype(int)
            frame2.append(df)
    decoy_val = [0]*len(frame2)
    
    #combining actives and decoys for data splitting
    frames = frame1+frame2
    x = pd.concat(frames)
    values = active_val+decoy_val
    y = pd.DataFrame(values)
    print(len(x), len(y))
    return x,y

In [15]:
def main(fname):
    activeSmiles(fname)
    print('extracted smiles for actives')
    decoySmiles(fname)
    print('extracted smiles for decoys')
    smiles2sdf(fname)
    TPATF(fname)
    x,y = getTPATF(fname)
    return x,y
x,y = main('Acetyl_coenzyme_A_transporter_1')

extracted smiles for actives
extracted smiles for decoys
generated sdf for actives
generated sdf for decoys
generated fingerprint for actives
generated fingerprint for decoys
processing actives features
processing decoys features
(14526, 14526)


In [16]:
#splitting the dataset into train and split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [20]:
#support vector classification

#t = (np.array(y_train).ravel())

classifier = SVC(kernel ='linear')
classifier.fit(x_train, y_train)
y_predicted = classifier.predict(x_test)
train_pred = classifier.predict(x_train)

train_score= roc_auc_score(y_train, train_pred)
test_score = roc_auc_score(y_test, y_predicted)

print('test:', test_score)
print('train:', train_score)

y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)


('test:', 1.0)
('train:', 1.0)
('sensitivity: ', 1.0)
('specificity: ', 1.0)
('PPV: ', 1.0)


In [21]:
#saving the model
pkl_filename = "/Users/dshrestha/Desktop/Pharos/models/AcetylCoA_transporter1.pkl"
pickle.dump(classifier, open(pkl_filename, 'wb'))
#with open(pkl_filename, 'wb') as file:
#    pickle.dump(classifier, file)

In [18]:
train_roc =[]
test_roc = []
test_sensitivity=[]
test_specificity=[]
test_PPV=[]
for n in range(20):
    print(n)
    x1_train, x1_test, y1_train, y1_test = train_test_split(x, y, test_size = 0.2)
    classifier1 = SVC(kernel ='linear')
    classifier1.fit(x1_train, y1_train)
    y1_predicted = classifier1.predict(x1_test)
    train1_pred = classifier1.predict(x1_train)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
    test_sensitivity.append(float(tp)/(tp+fn))
    test_specificity.append(float(tn)/(tn+fp))
    test_PPV.append(float(tp)/(tp+fp))
    train_roc.append(roc_auc_score(y1_train, train1_pred))
    test_roc.append(roc_auc_score(y1_test, y1_predicted))
print(np.mean(train_roc))
print(np.mean(test_roc))
print('sensitivity: ', np.mean(test_sensitivity))
print('specificity: ', np.mean(test_specificity))
print('PPV: ', np.mean(test_PPV))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1.0
0.999895742435
('sensitivity: ', 1.0)
('specificity: ', 1.0)
('PPV: ', 1.0)
